# Law, Bias, and Algorithms
## Included variable bias (1/2)

In this exercise, we will investigate how to use a regression model to measure disparities across different groups, and discuss some of the problems that might arise in doing so. We will use the NYC stop and frisk data we have been using in previous exercises.

In [1]:
# Some initial setup
options(digits = 3)
library(tidyverse)

theme_set(theme_bw())

# Read the data
stop_df <- read_rds("../data/sqf_sample.rds")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.3.2
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


The loaded data frame is a sample of stops in NYC, recorded on a 
[UF-250 form][uf250_link]

Below is a list of columns in the data, roughly corresponding to the [UF-250 form][uf250_link]:

* Base information regarding stop:
    * `id`, `year`, `date`, `time`, `precinct`, `location_housing`, 
      `suspected_crime`

* Circumstances which led to stop:
    * `stop_reason_object`, `stop_reason_desc`, `stop_reason_casing`,
      `stop_reason_lookout`, `stop_reason_clothing`, `stop_reason_drugs`,
      `stop_reason_furtive`, `stop_reason_violent`, `stop_reason_bulge`,
      `stop_reason_other` 
    
* Suspect demographics:
    * `suspect_dob`, `suspect_id_type`, `suspect_sex`, `suspect_race`,
      `suspect_hispanic`, `suspect_age`, `suspect_height`, `suspect_weight`,
      `suspect_hair`, `suspect_eye`, `suspect_build`, `reason_explained`,
      `others_stopped`

* Whether physical force was used:
    * `force_hands`, `force_wall`, `force_ground`, `force_drawn`,
      `force_pointed`, `force_baton`, `force_handcuffs`,
      `force_pepper`, `force_other`

* Was suspect arrested?: `arrested`

* Was summons issued?: `summons_issued`

* Officer in uniform?: `officer_uniform`, `officer_verbal`, `officer_shield`

* Was person frisked?: `frisked`
    * if yes: `frisk_reason_suspected_crime`, `frisk_reason_weapons`, 
      `frisk_reason_attire`, `frisk_reason_actual_crime`, 
      `frisk_reason_noncompliance`, `frisk_reason_threats`,
      `frisk_reason_prior`, `frisk_reason_furtive`, `frisk_reason_bulge`

* Was person searched?: `searched`,
    * if yes: `searched_hardobject`, `searched_outline`,
      `searched_admission`, `searched_other`

* Was weapon found?: `found_weapon`
    * if yes: `found_gun`, `found_pistol`, `found_rifle`, `found_assault`,
      `found_knife`, `found_machinegun`, `found_other`
      
* Was other contraband found?: `found_contraband`

* Additional circumstances/factors
    * `additional_report`, `additional_investigation`, `additional_proximity`, 
      `additional_evasive`, `additional_associating`, `additional_direction`, 
      `additional_highcrime`, `additional_time`, `additional_sights`, 
      `additional_other`

* Additional reports prepared: `extra_reports`

[uf250_link]: https://www.prisonlegalnews.org/media/publications/Blank%20UF-250%20Form%20-%20Stop%2C%20Question%20and%20Frisk%20Report%20Worksheet%2C%20NYPD%2C%202016.pdf

## Base rate disparities in the decision to frisk

First, let's measure the disparities in police decisions to frisk individuals of different race groups.

### Exercise 1: manual computation of odds and odds ratios

* **Step 1**: For each race group, compute the proportion that were frisked

In [2]:
# With the stop_df data, group by suspect_race and compute the proportion (mean) of frisked == 1
# WRITE CODE HERE
# START solution
p_frisked_df <- stop_df %>%
    group_by(suspect_race) %>%
    summarize(p_frisked = mean(frisked))
# END solution

* **Step 2**: Given probability $p$ of being frisked, the *odds* of being frisked is computed as $p / (1-p)$. 

For example, if $p = \frac{1}{2}$, you're equally likely to be frisked or not (i.e., odds = 1); if $p = \frac{2}{3}$, you're twice as likely to be frisked than not (odds = 2).

Using the proportion frisked from Step 1 as an estimate of the probability of being frisked, compute the *odds* of being frisked for each race.

In [3]:
# Compute the odds, p / (1-p), where p is the proportion from step 1
# WRITE CODE HERE
# START solution
odds_df <- p_frisked_df %>%
    mutate(odds = p_frisked / (1 - p_frisked))
# END solution

* **Step 3**: A common method of comparing odds between two groups is to compute the *odds ratio*. 
This is simply the ratio between two odds.

For example, if the odds of being frisked is 0.8 for white pedestrians and 1.6 for black pedestrians, the odds ratio of being frisked for black vs. white pedestrians would be $1.6 / 0.8 = 2$. In other words, we would say stopped black pedestrians have twice the odds of being frisked, compared to stopped white pedestrians.

Using the odds computed in Step 2, compute the odds ratio for minority groups (black / Hispanic) versus whites.

In [4]:
# Compute odds of frisk for minority race group / odds of frisk for whites
# WRITE CODE HERE
# START solution
# Purely-tidy solution
odds_df %>%
    select(suspect_race, odds) %>%
    spread(suspect_race, odds) %>%
    transmute(black_white_odds = black / white, hispanic_white_odds = hispanic / white)

# Alternative solution
odds_black <- odds_df$odds[odds_df$suspect_race == "black"]
odds_hispanic <- odds_df$odds[odds_df$suspect_race == "hispanic"]
odds_white <- odds_df$odds[odds_df$suspect_race == "white"]

cat("Black-white odds ratio:", odds_black / odds_white, "\n")
cat("Hispanic-white odds ratio:", odds_hispanic / odds_white)
# END solution

black_white_odds,hispanic_white_odds
2.1,1.88


Black-white odds ratio: 2.1 
Hispanic-white odds ratio: 1.88

### Base rate disparities with (logistic) regression

Another method for comparing differences in frisk rates is to use regression. 
Specifically, logistic regression is commonly used for binary decisions (e.g., where the decision is either "frisk" or "don't frisk").

In `R` we use the `glm` function to fit *generalized* linear models (e.g., logistic regression, poisson regression). 
In its simplest form, the `glm` function is specified with a `formula`, the `data`, and a `family` which indicates what type of regression is used.
A `formula` in `R` is specified in the form: `Left-hand-side variable ~ Right-hand-side specifications`.
For example, to fit a logistic regression (which is of the `"binomial"` family) of `frisked` to the `suspect_race` variable, using the `stop_df` data, we can write:

In [5]:
base_model <- glm(frisked ~ suspect_race, data = stop_df, family = binomial)

where the first argument to `glm` is assumed to be the `formula`. 

Using mathematical notation, this corresponds to the model:
$$
\Pr(\text{frisked}) = \operatorname{logit}^{-1}(
    \beta_0 + \beta_{\text{black}}\mathbb{1}_{\text{black}} + 
    \beta_{\text{Hispanic}}\mathbb{1}_{\text{Hispanic}}
),
$$
where
$$
\operatorname{logit}^{-1}(x) = 
\frac{e^{x}}{1 + e^{x}}
$$
and
$$
\operatorname{logit}(p) = \log\left(\frac{p}{1-p}\right).
$$

As a result
$$
\log\left(\frac{\Pr(\text{frisked})}
{1 - \Pr(\text{frisked})}\right) = 
\beta_0 + \beta_{\text{black}}\mathbb{1}_{\text{black}} + 
    \beta_{\text{Hispanic}}\mathbb{1}_{\text{Hispanic}},
$$
and so
$$
\frac{\Pr(\text{frisked})}{1 - \Pr(\text{frisked})}  = 
\exp\left(\beta_0 + \beta_{\text{black}}\mathbb{1}_{\text{black}} + 
    \beta_{\text{Hispanic}}\mathbb{1}_{\text{Hispanic}}\right).
$$

From the above model, $\exp(\beta_0)$ is the odds of being frisked for white individuals and $\exp(\beta_0+\beta_{\text{black}}) = \exp(\beta_0)\exp(\beta_{\text{black}})$ is the odds of being frisked for black individuals.
Consequently, the odds _ratio_ of being frisked for black vs. white pedestrians is $\exp(\beta_{\text{black}})$: the exponentiated coefficient of
the variable indicating whether a pedestrian's race group is black or not.

We can inspect the coefficients of the fitted model using the `coef()` function.

In [6]:
print(coef(base_model))

         (Intercept)    suspect_raceblack suspect_racehispanic 
               0.304                0.740                0.633 


As we've seen above, the `(Intercept)` ($\beta_0$) term corresponds to the _log_-odds of being frisked for stopped white individuals, while the `suspect_raceblack` coefficient represents the change in *log*-odds (log of odds ratio) of being frisked for black individuals compared to white individuals. By exponentiating the coefficients, we can recover the odds of being frisked for whites and odds-ratio of being frisked for each minority race group with respect to whites.

In [7]:
# Exponentiating the coefficients recover odds ratio of treatment for each variable; 
# identical to what we find in Exercise 1, 
# while the exponentiated intercept represents the odds of treatment for the base case (whites) 
print(exp(coef(base_model)))

         (Intercept)    suspect_raceblack suspect_racehispanic 
                1.36                 2.10                 1.88 


## Adjusting for other variables

One concern is that officers might have a legitimate reason to frisk certain individuals more often; it might just be that the "legitimate reason" is also highly correlated with race.

For example, as we discussed in week 2, one of the reasons for stopping an individual is if the officer suspects criminal posession of a weapon (encoded in the `suspected_crime` column as `cpw`).
Given that the primary justification of a frisk is concern for officer safety, one could argue that it is reasonable for an officer to 
frisk individuals whom they have stopped under suspicion of criminal posession of weapons.

(Although, whether an officer's _suspicion_ itself is justified is a different question, which we will address later)

### Adjusting for `suspected_crime == "cpw"` 

Given a regression model, we can "adjust for" additional variables in our data by including them in the right-hand side of our formula.

### Exercise 2: 

With `stop_df`, first create a new binary column named `is_cpw` that is `TRUE` if `suspected_crime` is `cpw`. Then fit a model that adjusts for this new covariate, and discuss the results.

In [13]:
# WRITE CODE HERE
# START solution

stop_df <- stop_df %>%
  mutate(is_cpw = suspected_crime == "cpw")

cpw_model <- glm(frisked ~ suspect_race + is_cpw, data = stop_df, family = binomial)
print(exp(coef(cpw_model)))

# END solution

         (Intercept)    suspect_raceblack suspect_racehispanic 
                1.13                 1.50                 1.59 
          is_cpwTRUE 
                9.76 


### Exercise 3: Adjusting for confounding

Following the above logic, there could be multiple legitimate factors that account for the observed disparity of being frisked between different race groups. 
Explore the effects of adjusting for more covariates on the results.

In [9]:
# Use race_coefs(m) to inspect just the race coefficients of any fitted model m
race_coefs <- function(m) {
    coef(m)[c("suspect_raceblack", "suspect_racehispanic")]
}

# WRITE CODE HERE
# START solution
# Example models including multiple variables
print(race_coefs(glm(frisked ~ suspect_race + suspected_crime + location_housing, 
               data = stop_df, family = binomial)))

print(race_coefs(glm(frisked ~ suspect_race + 
               suspected_crime + 
               location_housing + 
               precinct + 
               suspect_sex +
               suspect_age +
               stop_reason_object +
               stop_reason_furtive, 
               data = stop_df, family = binomial)))
# END solution

   suspect_raceblack suspect_racehispanic 
               0.240                0.324 
   suspect_raceblack suspect_racehispanic 
               0.207                0.149 


## A kitchen sink approach 

One common method for measuring disparities while addressing some of the omitted variable bias concerns is to include _all_ recorded data, that would have been available to the officer at the time of making the decision (to frisk an individual). This is also known as the "kitchen sink" approach.

The code below applies the kitchen sink approach to measure the disparate impact of 
frisk on minority race groups.

In [10]:
feats <- c(
    "suspected_crime",
    "precinct",
    "location_housing",
    "suspect_sex",
    "suspect_age",
    "suspect_height",
    "suspect_weight",
    "suspect_hair",
    "suspect_eye",
    "suspect_build",
    "additional_report",
    "additional_investigation",
    "additional_proximity",
    "additional_evasive",
    "additional_associating",
    "additional_direction",
    "additional_highcrime",
    "additional_time",
    "additional_sights",
    "additional_other",
    "stop_reason_object",
    "stop_reason_desc",
    "stop_reason_casing",
    "stop_reason_lookout",
    "stop_reason_clothing",
    "stop_reason_drugs",
    "stop_reason_furtive",
    "stop_reason_violent",
    "stop_reason_bulge",
    "stop_reason_other",
    "suspect_race"
)

# This creates a formula with a specified left-hand side (response = "frisked"),
# and using all the variables in feats on the right-hand side. 
# Constructing a formula in this way (instead of typing out all the variable names)
# is helpful for constructing multiple models that share a long list of variables in the right-hand side.
kitchen_sink_formula <- reformulate(feats, response = "frisked")

# We are only interested in the race coefficients
ks_model <- glm(kitchen_sink_formula, stop_df, family = binomial)
print(race_coefs(ks_model))

   suspect_raceblack suspect_racehispanic 
               0.191                0.173 


### Exercise 4:

Do you think this is a reasonable approach to measuring disparate impact?
What about disparate treatment?